In [402]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df1 = pd.read_csv("train.csv")
df2 = pd.read_csv("test.csv")

In [403]:
print(len(df1[df1["Survived"]==1].values)/891)
print(len(df1[(pd.isnull(df1["Cabin"]))&(df1["Survived"]==1)].values)/len(df1[pd.isnull(df1["Cabin"])].values))
print(len(df1[(pd.notnull(df1["Cabin"]))&(df1["Survived"]==1)].values)/len(df1[pd.notnull(df1["Cabin"])].values))

0.3838383838383838
0.29985443959243085
0.6666666666666666


In [404]:
#traindataを元に家族別の生存率を追加
name_df_train = df1.iloc[:,3]
survive_array = df1.iloc[:,1]

name_df = name_df_train

name_array = []
name_count = []
name_survive_count = []
name_survive_ratio = []

name_i = 0

for name in name_df:
    name_split = name.split(" ")
    name_split_first = name_split[0].replace(',','')
    if name_split_first in name_array:
        name_count[name_array.index(name_split_first)] = name_count[name_array.index(name_split_first)] + 1
        name_survive_count[name_array.index(name_split_first)] = name_survive_count[name_array.index(name_split_first)] + survive_array[name_i]                            
    else:
        name_array.append(name_split_first)
        name_count.append(1)
        name_survive_count.append(survive_array[name_i])
    name_i = name_i + 1
        
count = 0
for i in name_count:
    if name_count[count] == 1:
        name_survive_ratio.append(-1)
    else:
        name_survive_ratio.append(name_survive_count[count]/(name_count[count]))
    count = count + 1
    
survive_ratio_array = []    

for name in name_df:
    name_split = name.split(" ")
    name_split_first = name_split[0].replace(',','')
    survive_ratio_array.append(name_survive_ratio[name_array.index(name_split_first)])

df1['familysurviveratio'] = np.array(survive_ratio_array)

In [405]:
#testdataにも家族別生存率を追加
name_df_test = df2.iloc[:,2]
survive_ratio_test =[]
for name in name_df_test:
    name_split = name.split(" ")
    name_split_first = name_split[0].replace(',','')                                         
    if name_split_first in name_array:
        survive_ratio_test.append(name_survive_ratio[name_array.index(name_split_first)])
    else:
        survive_ratio_test.append(-1)
df2= df2.drop(["Ticket"], axis = 1)        
df2['familysurviveratio'] = np.array(survive_ratio_test)

In [406]:
#1つのデータフレームにして処理を行う
df =  pd.concat([df1,df2])

In [407]:
df = df.reset_index(drop=True)

In [408]:
#敬称を追加
honor = []
for i in df["Name"]:
    if "Master" in i :
        honor.append(0)
    elif "Mr" in i and "Mrs" not in i:
        honor.append(1)
    elif "Miss" in i :
        honor.append(2)
    elif "Mrs" in i :
        honor.append(3)
    else:
        honor.append(4)
    
df['honor'] = np.array(honor)

In [409]:
#家族の人数を追加
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1

In [410]:
#家族の人数に応じたラベルを追加
def familysizerange(FamilySize):
    if FamilySize == 1:
        return 0
    elif FamilySize ==2 or FamilySize == 3:
        return 1
    elif FamilySize >= 4:
        return 2
    
df["familysizerange"] = df.FamilySize.apply(familysizerange)

In [411]:
def Cabinisnull(Cabin):
    if pd.isnull(Cabin):
        return 0
    else:
        return 1
    
df["Cabinisnull"] = df.Cabin.apply(Cabinisnull)

In [412]:
#ダミー変数化
df = df.replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)

In [413]:
#年齢の欠損を敬称、性別、Pclassの一致する人の中央値で埋める。
df = df.fillna(value = {"Age":-1})
a = df.loc[:,["Age","honor","Pclass","Sex"]].values
for i in a:
    if i[0] == -1:
        i[0] =df[(df["honor"]==i[1])&(df["Pclass"]==i[2])&(df["Sex"]==i[3])]["Age"].median()
frame = pd.DataFrame(a, columns=["Age", "honor","Pclass","Sex"])
df.drop(["Age","honor","Pclass","Sex"],axis = 1,inplace = True)
df = pd.concat([df, frame], axis=1)

In [414]:
#家族の人数に応じたラベルを追加
def familysizerange(FamilySize):
    if FamilySize == 1:
        return 0
    elif FamilySize ==2 or FamilySize == 3:
        return 1
    elif FamilySize >= 4:
        return 2
    
df["familysizerange"] = df.FamilySize.apply(familysizerange)

In [415]:
#Ageを5歳幅に区切る。
def agerange(Age):
    return int(Age/5) 

df["agerange"] = df.Age.apply(agerange)
df.drop(["Age"],axis = 1,inplace  =True)

In [416]:
# 学習用にsurvivedを取り出しておく。取り出したらSurvivedは消す。
y = df["Survived"][:890].as_matrix()
df.drop(["Survived"],axis = 1,inplace = True)

In [417]:
#欠損値を埋める(ほぼ無いので適当に中央値で埋める)
df = df.fillna(df.median())

In [418]:
df.count()

Cabin                  295
Embarked              1309
Fare                  1309
Name                  1309
Parch                 1309
PassengerId           1309
SibSp                 1309
Ticket                 891
familysurviveratio    1309
FamilySize            1309
familysizerange       1309
Cabinisnull           1309
honor                 1309
Pclass                1309
Sex                   1309
agerange              1309
dtype: int64

In [419]:
#使わない情報を落とす
df.drop(["Cabin","Parch","SibSp","Ticket","FamilySize","Name","PassengerId"],axis =1,inplace = True)

In [420]:
df.head()

,Embarked,Fare,familysurviveratio,familysizerange,Cabinisnull,honor,Pclass,Sex,agerange
0,0.0,7.2500,0.0,1,0,1.0,3.0,0.0,4
1,1.0,71.2833,-1.0,1,1,3.0,1.0,1.0,7
2,0.0,7.9250,-1.0,0,0,2.0,3.0,1.0,5
3,0.0,53.1000,0.5,1,1,3.0,1.0,1.0,7
4,0.0,8.0500,0.5,0,0,1.0,3.0,0.0,7


In [421]:
#traindataの取り出し
X = df.iloc[:,:][:890].values

In [422]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [425]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
mod = xgb.XGBClassifier()
mod.fit(X_train, y_train)
print('train accuracy: %.3f' % mod.score(X_train, y_train))
print('test  accuracy: %.3f' % mod.score(X_test, y_test))

train accuracy: 0.907
test  accuracy: 0.846


In [426]:
#testdataの取り出し
x1 = df.iloc[:,:][891:].values
#予測
y1 = mod.predict(x1)

In [427]:
#csvデータに書き出す。
a =[i for i in range(892,1310)]
dfnew2 = pd.DataFrame({'PassengerId':a ,'Survived': y1})
dfnew2.to_csv("pred47.csv", index = False)